In [1]:
import os
import numpy as np

from rmgpy.tools.uncertainty import Uncertainty, process_local_results
from rmgpy.tools.canteramodel import get_rmg_species_from_user_species
from rmgpy.species import Species

## Step 1: Define mechanism files

In [2]:
# Must use annotated chemkin file
chemkin_file = './data/parse_source/chem_annotated.inp'
dict_file = './data/parse_source/species_dictionary.txt'

# Initialize the Uncertainty class instance and load the model
uncertainty = Uncertainty(output_directory='.temp/uncertainty')
uncertainty.load_model(chemkin_file, dict_file)


## Kinetics Uncertainty for Rate Rules


The rate $\ln k_\mathrm{site}$ can be written as a sum of other random variables, as shown in the block diagram above.

$$\ln k_\mathrm{site} = \left[\sum_{rule_i}\omega_i \ln(k_\mathrm{rule,i})\right] + \ln k_\mathrm{family}+\log {10}(N+1)\ln k_\mathrm{non-exact}$$


If we consider each of $\ln(k_\mathrm{rule,i})$, $\ln(k_\mathrm{family})$, and $\ln(k_\mathrm{non-exact})$ to be independent random variables, then we can derive the total variance $\sigma^2(\ln k_\mathrm{site})$ using the following [property](https://en.wikipedia.org/wiki/Variance#Basic_properties):

$$\sigma^2(aX+bY)=a^2\sigma^2(X)+b^2\sigma^2(Y)+2abcov(X,Y)$$

$\ln(k_\mathrm{rule,i})$, $\ln(k_\mathrm{family})$, and $\ln(k_\mathrm{non-exact})$ are all mutually independent, so the $2abcov(X,Y)$ term drops out in every case.

$$\sigma^2(\ln k_\mathrm{site}) = \left[\sum_{rule_i}\omega_i^2 \sigma^2(\ln(k_\mathrm{rule,i}))\right] + \sigma^2(\ln k_\mathrm{family})+\left[\log {10}(N+1)\right]^2\sigma^2(\ln k_\mathrm{non-exact})$$



what is currently implemented:
$$\sigma(\ln k_\mathrm{site}) = \left[\sum_{rule_i}\omega_i \sigma(\ln(k_\mathrm{rule,i}))\right] + \sigma(\ln k_\mathrm{family})+\left[\log {10}(N+1)\right]\sigma(\ln k_\mathrm{non-exact})$$


### Covariance Matrix
We want to compute elements of the covariance matrix $\Sigma_{ab}=cov(\ln(k_{rate\ rules,a}), \ln(k_{rate\ rules,b}))$

$$\Sigma_{ab}=cov(\ln(k_{site,a}), \ln(k_{site,b}))$$

$$\Sigma_{ab}=cov\left(\left[\sum_{rule_i}\omega_{a,i} \ln(k_\mathrm{rule,i})\right] + \ln k_\mathrm{family,a}+\log {10}(N_a+1)\ln k_\mathrm{non-exact,a}, \left[\sum_{rule_j}\omega_{b,j} \ln(k_\mathrm{rule,j})\right] + \ln k_\mathrm{family,b}+\log {10}(N_b+1)\ln k_\mathrm{non-exact,b}\right)$$

The [covariance of linear combinations](https://en.wikipedia.org/wiki/Covariance#Covariance_of_linear_combinations) is the following:
$$cov(aX+bY, cV+dW)=ac\cdot cov(X,V)+ad\cdot cov(X,W)+bc\cdot cov(Y,V)+bd\cdot cov(Y,W)$$

Or, more generally:
$$cov(\sum_ia_iX_i, \sum_jb_jY_j)=\sum_{i}\sum_{j}a_ib_j cov(X_i, Y_j)$$


Before trying to write this all out, we know that the $\ln(k_\mathrm{rule,i})$, $\ln(k_\mathrm{family})$, and  $\ln(k_\mathrm{non-exact})$ are independent, so we have the following:


$$cov(\ln(k_\mathrm{rule,i}), \ln(k_\mathrm{family}))=0$$
$$cov(\ln(k_\mathrm{rule,i}), \ln(k_\mathrm{non-exact}))=0$$
$$cov(\ln(k_\mathrm{family}), \ln(k_\mathrm{non-exact}))=0$$


$$\Sigma_{ab}=\left[\sum_{rule_i}\sum_{rule_j}\omega_{a,i}\omega_{b,j} cov(\ln(k_\mathrm{rule,i}),\ln(k_\mathrm{rule,j}))\right]+cov(\ln k_\mathrm{family,a},\ln k_\mathrm{family,b})+\log {10}(N_a+1)\log {10}(N_b+1)cov(\ln k_\mathrm{non-exact,a},\ln k_\mathrm{non-exact,b})$$


### Check the diagonal to see if this matches the variance
$$\Sigma_{aa}=\left[\sum_{rule_i}\sum_{rule_j}\omega_{a,i}\omega_{a,j} cov(\ln(k_\mathrm{rule,i}),\ln(k_\mathrm{rule,j}))\right]+cov(\ln k_\mathrm{family,a},\ln k_\mathrm{family,a})+\log {10}(N_a+1)\log {10}(N_a+1)cov(\ln k_\mathrm{non-exact,a},\ln k_\mathrm{non-exact,a})$$


$$\Sigma_{aa}=\left[\sum_{rule_i}\sum_{rule_j}\omega_{a,i}\omega_{a,j} cov(\ln(k_\mathrm{rule,i}),\ln(k_\mathrm{rule,j}))\right]+\sigma^2(\ln k_\mathrm{family,a})+\left[\log {10}(N_a+1)\right]^2\sigma^2(\ln k_\mathrm{non-exact,a})$$


Different rate rules are independent from each other, so $cov(\ln(k_\mathrm{rule,i}),\ln(k_\mathrm{rule,j}))=0$ for $i \neq j$


$$\Sigma_{aa}=\left[\sum_{rule_i}\omega_{a,i}^2 \sigma^2(\ln(k_\mathrm{rule,i}))\right]+\sigma^2(\ln k_\mathrm{family,a})+\left[\log {10}(N_a+1)\right]^2\sigma^2(\ln k_\mathrm{non-exact,a})$$

In [3]:
# NOTE: You must load the database with the same settings which were used to generate the model.
#       This includes any thermo or kinetics libraries which were used. This takes a while.
uncertainty.load_database(
    thermo_libraries=['primaryThermoLibrary'],
    kinetics_families='default',
    reaction_libraries=[],
)


In [4]:
# Calculate the covariance matrix, then split it up into separate matrices for kinetic and thermo parameters
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties(correlated=True)
Sigma, labels = uncertainty.get_uncertainty_covariance_matrix()
N = len(uncertainty.species_list)
Sigma_G = Sigma[0:N,0:N]
Sigma_k = Sigma[N:,N:]

In [5]:
# Walk through variance example reaction 21,21

# Display the source information for estimating reaction 21's rate
a=21
reaction_a = uncertainty.reaction_list[a]
print(f'Reaction {a}:\t', reaction_a)
print()
print('Source Dictionary')
print(uncertainty.reaction_sources_dict[reaction_a])
print()
print(uncertainty.reaction_sources_dict[reaction_a]['Rate Rules'][0])
training_rules=uncertainty.reaction_sources_dict[reaction_a]['Rate Rules'][1]['training']
for rule in training_rules:
    print(rule)

print()

print(f'Covariance Uncertainty [{a},{a}]:', Sigma_k[a,a])

# Hand calculation
variance_by_hand = 2.0 * (np.float_power(0.5, 2.0) * 0.5) + 1.0 + np.float_power(np.log10(3), 2.0) * 3.5
print(f'Hand calculated variance:\t{variance_by_hand}')
print()

# Check against the existing (modified) uncertainty code using the correlated option
total_uncertainty = 0
for key in uncertainty.kinetic_input_uncertainties[a]:
    total_uncertainty += uncertainty.kinetic_input_uncertainties[a][key]
print(f'Existing uncertainty code:\t{total_uncertainty}')
print()
print(uncertainty.kinetic_input_uncertainties[a])

Reaction 21:	 H(6) + [CH]=C(14) <=> H2(13) + C#C(28)

Source Dictionary
{'Rate Rules': ['Disproportionation', {'template': [<Entry index=22 label="H_rad">, <Entry index=208 label="Cds/H2_d_Crad">], 'degeneracy': 2.0, 'exact': False, 'rules': [], 'training': [(<Entry index=673 label="H_rad;Cds/H2_d_N3rad">, <Entry index=116 label="H + CH2N <=> H2 + CHN">, 0.5), (<Entry index=688 label="H_rad;Cds/H2_d_N5dcrad/O">, <Entry index=131 label="H + CH2NO <=> H2 + CHNO">, 0.5)]}]}

Disproportionation
(<Entry index=673 label="H_rad;Cds/H2_d_N3rad">, <Entry index=116 label="H + CH2N <=> H2 + CHN">, 0.5)
(<Entry index=688 label="H_rad;Cds/H2_d_N5dcrad/O">, <Entry index=131 label="H + CH2NO <=> H2 + CHNO">, 0.5)

Covariance Uncertainty [21,21]: 2.0467564209684275
Hand calculated variance:	2.0467564209684275

Existing uncertainty code:	2.0467564209684275

{'Disproportionation H_rad;Cds/H2_d_N3rad': 0.125, 'Disproportionation H_rad;Cds/H2_d_N5dcrad/O': 0.125, 'Estimation H(6)+C2H3(14)=H2(13)+C#C(28)':

In [6]:
# Thermo example
i=11
print(f'Species:\t{uncertainty.species_list[i]}')
print()
print('Uncertainty sources dictionary:')
print(uncertainty.species_sources_dict[uncertainty.species_list[i]])


print()
total_uncertainty = 0
for key in uncertainty.thermo_input_uncertainties[i].keys():
    total_uncertainty += uncertainty.thermo_input_uncertainties[i][key]
print(f'Existing uncertainty code:\t{total_uncertainty}')
print(uncertainty.thermo_input_uncertainties[i])
print()

# Hand calculation
variance_by_hand = np.float_power(2.0, 2.0)*0.1 + np.float_power(1.0, 2.0)*0.1 + 1.5
print(f'Hand calculated variance:\t{variance_by_hand}')

# Using the get_uncertainty_covariance_matrix function to estimate uncertainty
print(f'Covariance Uncertainty [{i},{i}]:', Sigma_G[i,i])


Species:	[CH]=C(14)

Uncertainty sources dictionary:
{'GAV': {'group': [(<Entry index=296 label="Cds-CdsHH">, 2)], 'radical': [(<Entry index=435 label="Cds_P">, 1)]}}

Existing uncertainty code:	2.0
{'Group(group) Cds-CdsHH': 0.4, 'Group(radical) Cds_P': 0.1, 'Estimation C2H3(14)': 1.5}

Hand calculated variance:	2.0
Covariance Uncertainty [11,11]: 2.0


In [7]:
# Walk through example 19,21

a=19
reaction_a = uncertainty.reaction_list[a]
print(f'Reaction {a}:\t', reaction_a)

print(uncertainty.reaction_sources_dict[reaction_a])

print()
b=21
reaction_b = uncertainty.reaction_list[b]
print(f'Reaction {b}:\t', str(reaction_b))
print(uncertainty.reaction_sources_dict[reaction_b])


print(f'Covariance Uncertainty [{a},{b}]: {Sigma_k[a, b]}')
#print('uncertainty.uncertainty', uncertainty.kinetic_input_uncertainties[a])
# the possible sources of the reactions


print(0.5*(0.5*0.1)+0.5*(0.5*0.1625))

Reaction 19:	 [CH3](4) + [CH]=C(14) <=> CH4(3) + C#C(28)
{'Rate Rules': ['Disproportionation', {'template': [<Entry index=59 label="C_methyl">, <Entry index=208 label="Cds/H2_d_Crad">], 'degeneracy': 2.0, 'exact': False, 'rules': [], 'training': [(<Entry index=673 label="H_rad;Cds/H2_d_N3rad">, <Entry index=116 label="H + CH2N <=> H2 + CHN">, 0.1), (<Entry index=688 label="H_rad;Cds/H2_d_N5dcrad/O">, <Entry index=131 label="H + CH2NO <=> H2 + CHNO">, 0.16250000000000003), (<Entry index=672 label="O_pri_rad;Cds/H2_d_N3rad">, <Entry index=115 label="HO + CH2N <=> H2O + CHN">, 0.05), (<Entry index=689 label="O_pri_rad;Cds/H2_d_N5dcrad/O">, <Entry index=132 label="HO + CH2NO <=> H2O + CHNO">, 0.1375), (<Entry index=669 label="O_rad/NonDeO;Cds/H2_d_N3rad">, <Entry index=112 label="HO2-2 + CH2N <=> H2O2 + CHN">, 0.07500000000000001), (<Entry index=674 label="NH2_rad;Cds/H2_d_N3rad">, <Entry index=117 label="H2N + CH2N <=> H3N + CHN">, 0.1), (<Entry index=691 label="NH2_rad;Cds/H2_d_N5dcrad/O

In [8]:
## Basic sanity checks on the covariance matrix

# check that the diagonal matches the uncorrelated variances

uncorrelated_matches_diagonal = True
uncertainty.assign_parameter_uncertainties(correlated=False)
for i in range(0, len(uncertainty.kinetic_input_uncertainties)):
    if uncertainty.kinetic_input_uncertainties[i] != Sigma_k[i,i]:
        print(uncertainty.kinetic_input_uncertainties[i], Sigma_k[i,i])
        uncorrelated_matches_diagonal = False

if uncorrelated_matches_diagonal:
    print("The diagonal of the covariance matrix matches the uncorrelated variances")

    
# check that the off diagonals are always less than the corresponding diagonals
diagonal_is_max = True
for row in range (0, Sigma_k.shape[0]): # horizontal
    for col in range(0, row): # col
        if Sigma_k[row, col] > Sigma_k[row,row]:
            diagonal_is_max = False
            print("Off-diagonal is too large!")
for col in range (0, Sigma_k.shape[0]): # vertical
    for row in range(col+1, Sigma_k.shape[0]): # col
        if Sigma_k[row, col] > Sigma_k[col,col]:
            diagonal_is_max = False
            print("Off-diagonal is too large!")     
            
            
if diagonal_is_max:
    print("All off-diagonals are less than diagonal")

        
    
# check that the covariance matrix is symmetric
is_symmetric = (Sigma_k == Sigma_k.transpose())
if is_symmetric.all():
    print("Covariance matrix is symmetric")
    

#check the covariance matrix is positive semidefinite
pos_semidef = True
for i in range(0, 100):
    x = np.random.rand(Sigma_k.shape[0]) - 0.5
    prod = np.dot(np.dot(Sigma_k,x), x)
    if prod < 0:
        print("Covariance matrix is not positive semi-definite")
        pos_semidef = False
if pos_semidef:
    print("Covariance matrix is positive semi-definite")


The diagonal of the covariance matrix matches the uncorrelated variances
All off-diagonals are less than diagonal
Covariance matrix is symmetric
Covariance matrix is positive semi-definite


In [9]:
# convert the matrix to sparse form because it's big
import pickle
from scipy.sparse import coo_matrix

sparse_cov_mat = coo_matrix(Sigma)
data = {
    'covariance_matrix': sparse_cov_mat,
    'parameter_labels': labels
}

with open('covariance_matrix.pickle', 'wb') as f:
    pickle.dump(data, f)


In [10]:
#List species and sources
for species in uncertainty.species_list:
    print(species)
    print(uncertainty.species_sources_dict[species])

Ar
{'Library': 0}
He
{'Library': 1}
Ne
{'Library': 2}
N2
{'Library': 3}
ethane(1)
{'GAV': {'group': [(<Entry index=901 label="Cs-CsHHH">, 2)]}}
CH4(3)
{'Library': 5}
[CH3](4)
{'Library': 5, 'GAV': {'radical': [(<Entry index=94 label="CH3">, 1)]}}
C[CH2](5)
{'GAV': {'group': [(<Entry index=901 label="Cs-CsHHH">, 2)], 'radical': [(<Entry index=117 label="CCJ">, 1)]}}
H(6)
{'Library': 8}
C=C(9)
{'GAV': {'group': [(<Entry index=296 label="Cds-CdsHH">, 2)]}}
H2(13)
{'Library': 10}
[CH]=C(14)
{'GAV': {'group': [(<Entry index=296 label="Cds-CdsHH">, 2)], 'radical': [(<Entry index=435 label="Cds_P">, 1)]}}
[CH2]CC(16)
{'GAV': {'group': [(<Entry index=933 label="Cs-CsCsHH">, 1), (<Entry index=901 label="Cs-CsHHH">, 2)], 'radical': [(<Entry index=118 label="RCCJ">, 1)]}}
C#C(28)
{'GAV': {'group': [(<Entry index=65 label="Ct-CtH">, 2)]}}
[CH2]CC=C(31)
{'GAV': {'group': [(<Entry index=937 label="Cs-(Cds-Cds)CsHH">, 1), (<Entry index=901 label="Cs-CsHHH">, 1), (<Entry index=325 label="Cds-CdsCsH">,

In [11]:
#print the list of reactions, sources, uncertainties:
uncertainty.assign_parameter_uncertainties(correlated=False)
for i in range(0,len(uncertainty.reaction_list)):
    print(i, uncertainty.reaction_list[i], uncertainty.kinetic_input_uncertainties[i])
    print(uncertainty.reaction_sources_dict[uncertainty.reaction_list[i]])
    print()
    

0 [CH3](4) + [CH3](4) <=> ethane(1) 0.5
{'Training': ['R_Recombination', <Entry index=9 label="CH3 + CH3 <=> C2H6">, False]}

1 [CH3](4) + ethane(1) <=> CH4(3) + C[CH2](5) 0.5
{'Training': ['H_Abstraction', <Entry index=215 label="C2H6 + CH3_r3 <=> C2H5b + CH4">, False]}

2 H(6) + C[CH2](5) <=> ethane(1) 0.5
{'Training': ['R_Recombination', <Entry index=58 label="H + C2H5 <=> C2H6-2">, False]}

3 H(6) + [CH3](4) <=> CH4(3) 0.5
{'Training': ['R_Recombination', <Entry index=57 label="H + CH3 <=> CH4">, False]}

4 H(6) + C[CH2](5) <=> ethane(1) 0.5
{'Training': ['R_Recombination', <Entry index=58 label="H + C2H5 <=> C2H6-2">, False]}

5 H(6) + C=C(9) <=> C[CH2](5) 0.5
{'Training': ['R_Addition_MultipleBond', <Entry index=2541 label="H + C2H4 <=> C2H5-2">, False]}

6 [CH3](4) + C[CH2](5) <=> CH4(3) + C=C(9) 0.5
{'Training': ['Disproportionation', <Entry index=5 label="CH3 + C2H5 <=> CH4 + C2H4">, False]}

7 C[CH2](5) + C[CH2](5) <=> C=C(9) + ethane(1) 0.5
{'Training': ['Disproportionation'